In [145]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

## Métodos Auxiliares

In [277]:
def showImage(imageTitle, image, waitForKey):
    cv2.imshow(imageTitle, image)
    
    if waitForKey:
        cv2.waitKey()
        cv2.destroyAllWindows()
    
def showImageArray(tituloArray, arrayImagens):
    for index in range(len(arrayImagens)):
        if index == len(arrayImagens) - 1:
            showImage(tituloArray + ": " + str(index), arrayImagens[index], True)
        else:
            showImage(tituloArray + ": " + str(index), arrayImagens[index], False)
            
def classificadorMoedas(areaMoeda):
    
    #Moeda 0.01
    if areaMoeda >= 5600 and areaMoeda <= 6000:
        return moedas.get("Moeda 0.01"), 0.01
    
    #Moeda 0.02
    if areaMoeda >= 8400 and areaMoeda <= 9000:
        return moedas.get("Moeda 0.02"), 0.02
    
    #Moeda 0.05
    if areaMoeda >= 11000 and areaMoeda <= 12000:
        return moedas.get("Moeda 0.05"), 0.05
    
    #Moeda 0.10
    if areaMoeda >= 9400 and areaMoeda <= 10500:
        return moedas.get("Moeda 0.10"), 0.10
    
    #Moeda 0.20
    if areaMoeda >= 12500 and areaMoeda <= 13500:
        return moedas.get("Moeda 0.20"), 0.20
    
    #Moeda 0.50
    if areaMoeda >= 15800 and areaMoeda <= 16700:
        return moedas.get("Moeda 0.50"), 0.50
    
    #Moeda 1.00
    if areaMoeda >= 14100 and areaMoeda <= 15300:
        return moedas.get("Moeda 1.00"), 1.0
    
    #Moeda 2.00
    if areaMoeda >= 17000 and areaMoeda <= 18600:
        return moedas.get("Moeda 2.00"), 2.0

## Carregar Imagens

In [147]:
numberOfImages = 9
imagensOriginais = [None] * numberOfImages
initialIndex = 1000697
currentIndex = initialIndex

for index in range(numberOfImages):
    while imagensOriginais[index] is None:
        imagensOriginais[index] = cv2.imread("P" + str(currentIndex) + "s.jpg")
        currentIndex = currentIndex + 1

## Obter o Canal RED das Imagens

In [148]:
imagensCanalRED = [None] * numberOfImages

for index in range(numberOfImages):
    imagensCanalRED[index] = imagensOriginais[index][:, :, 2]

## Obter Thresholds e Imagens Binárias

In [149]:
imagensBinarias = [None] * numberOfImages
imagensThresholds = [None] * numberOfImages

for index in range(numberOfImages):
    imagensThresholds[index], imagensBinarias[index] = cv2.threshold(imagensCanalRED[index], 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

In [150]:
print(imagensThresholds)

[134.0, 140.0, 142.0, 117.0, 144.0, 144.0, 142.0, 119.0, 144.0]


In [151]:
imagensBinarias = [None] * numberOfImages
imagensThresholds = [None] * numberOfImages

for index in range(numberOfImages):
    imagensThresholds[index], imagensBinarias[index] = cv2.threshold(imagensCanalRED[index], 100, 255, cv2.THRESH_BINARY)

## Elementos Estruturantes

In [152]:
maskClose = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
maskErode = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
maskDilate = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))

## Aplicação das Operações Morfológicas

In [279]:
imagensPosOperacoesMorfologicas = [None] * numberOfImages

for index in range(numberOfImages):

    imagensPosOperacoesMorfologicas[index] = cv2.morphologyEx(imagensBinarias[index], cv2.MORPH_CLOSE, maskDilate)
    imagensPosOperacoesMorfologicas[index] = cv2.erode(imagensPosOperacoesMorfologicas[index], maskErode, iterations=7)    
    imagensPosOperacoesMorfologicas[index] = cv2.dilate(imagensPosOperacoesMorfologicas[index], maskDilate, iterations=7)

## Criação das Máscaras das Moedas

In [284]:
valorMoedasOrdenado = [0.01, 0.02, 0.05, 0.10, 0.20, 0.50, 1.0, 2.0]
moedas = {}

for index in range(len(valorMoedasOrdenado)):
    moeda = {"Moeda " + str(f"{valorMoedasOrdenado[index]:.2f}") : valorMoedasOrdenado[index]}
    
    moedas.update({"Moeda " + str(f"{valorMoedasOrdenado[index]:.2f}") : moeda})

In [285]:
moedas

{'Moeda 0.01': {'Moeda 0.01': 0.01},
 'Moeda 0.02': {'Moeda 0.02': 0.02},
 'Moeda 0.05': {'Moeda 0.05': 0.05},
 'Moeda 0.10': {'Moeda 0.10': 0.1},
 'Moeda 0.20': {'Moeda 0.20': 0.2},
 'Moeda 0.50': {'Moeda 0.50': 0.5},
 'Moeda 1.00': {'Moeda 1.00': 1.0},
 'Moeda 2.00': {'Moeda 2.00': 2.0}}

## Classficação das Moedas nas Imagens

In [286]:
valorImagens = [None] * numberOfImages
valorTotalImagens = np.zeros(numberOfImages)

for index in range(numberOfImages):
    numLabels, labels, stats, centroids = cv2.connectedComponentsWithStats(imagensPosOperacoesMorfologicas[index], 8, cv2.CV_32S)

    valorImagem = {}
    valorTotal = 0
    
    for i in range(numLabels):
        
        if i != 0:
            
            area = stats[i, cv2.CC_STAT_AREA]            
            valor = classificadorMoedas(area)
                
            if valor is not None:
                valorImagem.update({"moeda " + str(i - 1) : valor[0]})
                valorTotal = valorTotal + valor[1]
    
    valorImagens[index] = valorImagem
    valorTotalImagens[index] = valorTotal

## Comparação com Valores Reais

In [287]:
valoresReais = [2.08, 1.33, 0.78, 0.67, 1.62, 1.67, 1.30, 1.30, 1.70]

for index in range(numberOfImages):
    print(valoresReais[index], valorTotalImagens[index])

2.08 2.08
1.33 1.33
0.78 0.78
0.67 0.67
1.62 1.62
1.67 1.67
1.3 1.3
1.3 1.3
1.7 1.7


## Cálculo para Moeda de 2 Euros

In [160]:
# Calculo da Area Real das Moedas
areaCentimo1 = np.pi * (16.25/2)**2 #1 Centimo
areaCentimo2 = np.pi * (18.75/2)**2 #2 Centimos
areaCentimo5 = np.pi * (21.25/2)**2 #5 Centimos
areaCentimo10 = np.pi * (19.75/2)**2 #10 Centimos
areaCentimo20 = np.pi * (22.25/2)**2 #20 Centimos
areaCentimo50 = np.pi * (24.25/2)**2 #50 Centimos
areaEuro1 = np.pi * (23.25/2)**2 #1 Euro
areaEuro2 = np.pi * (25.75/2)**2 #2 Euro

# Area Minima das Moedas
areaMinimaEuro2 = np.zeros(7)

areaMinimaEuro2[0] = areaEuro2 * 5727. / areaCentimo1
areaMinimaEuro2[1] = areaEuro2 * 8581. / areaCentimo2
areaMinimaEuro2[2] = areaEuro2 * 11305. / areaCentimo5
areaMinimaEuro2[3] = areaEuro2 * 9557. / areaCentimo10
areaMinimaEuro2[4] = areaEuro2 * 12796. / areaCentimo20
areaMinimaEuro2[5] = areaEuro2 * 16056. / areaCentimo50
areaMinimaEuro2[6] = areaEuro2 * 14456. / areaEuro1

# Area Maxima das Moedas
areaMaximaEuro2 = np.zeros(7)

areaMaximaEuro2[0] = areaEuro2 * 5910. / areaCentimo1
areaMaximaEuro2[1] = areaEuro2 * 8842. / areaCentimo2
areaMaximaEuro2[2] = areaEuro2 * 11731. / areaCentimo5
areaMaximaEuro2[3] = areaEuro2 * 10078. / areaCentimo10
areaMaximaEuro2[4] = areaEuro2 * 13444. / areaCentimo20
areaMaximaEuro2[5] = areaEuro2 * 16434. / areaCentimo50
areaMaximaEuro2[6] = areaEuro2 * 14941. / areaEuro1

#Arranjar Minimos e Máximos para a Moeda de 2 Euros
minimoEuro2 = np.min(areaMinimaEuro2[areaMinimaEuro2 > 16700])
maximoEuro2 = np.max(areaMaximaEuro2[(areaMaximaEuro2 > minimoEuro2) & (areaMaximaEuro2 < 18600)])

In [161]:
print(minimoEuro2, maximoEuro2)

17138.336573664943 18529.95068551387
